# Modelo em produção

# Gerando um dataset e treinando um modelo

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np

In [2]:
rng = np.random.RandomState(0)
X, y = make_classification(n_samples=1000, n_features = 5, random_state=rng)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Ajustando um random forest

In [4]:
clf = RandomForestClassifier(n_estimators=10, random_state=rng)
model = clf.fit(X_train, y_train)

In [5]:
pred = model.predict(X_test)

In [6]:
from sklearn.metrics import classification_report, recall_score, precision_score
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96        99
           1       0.96      0.97      0.97       101

    accuracy                           0.96       200
   macro avg       0.97      0.96      0.96       200
weighted avg       0.97      0.96      0.96       200



In [7]:
model_metrics = dict()
model_metrics['recall'] = recall_score(y_test, pred)
model_metrics['precision'] = precision_score(y_test, pred)

In [8]:
model_metrics

{'recall': 0.9702970297029703, 'precision': 0.9607843137254902}

In [9]:
#caso não tenha criado a pasta metrics, execute o código a seguir
!mkdir metrics
import json

with open('metrics/model_metrics_version_clf.json', 'w') as fp:
    json.dump(model_metrics, fp)

mkdir: cannot create directory ‘metrics’: File exists


## Salvando modelo

In [10]:
from joblib import dump, load

In [11]:
#caso não tenha criado a pasta models, execute o código a seguir
!mkdir models

dump(clf, 'models/clf.joblib') 

mkdir: cannot create directory ‘models’: File exists


['models/clf.joblib']

# Criando API

In [12]:
from flask import Flask, request, url_for, redirect, render_template, jsonify
import numpy as np
import pandas as pd

In [13]:
app = Flask(__name__)

In [14]:
model = load('models/clf.joblib')

In [15]:
@app.route('/')
def home():
    return render_template('templates/template.html')

In [16]:
@app.route('/predict', methods = ['POST'])
def predict():
    #request of all inputs
    features = [x for x in request.form.values()]
    
    #data preparing
    features_array = np.array(features)
    pandas_features = pd.DataFrame([features_array])
    print(features)
    
    #predict
    pred = model.predict(pandas_features)
    print(pred)
    prediction = int(pred[0])
    return render_template("home.html", pred = "prediction: {}".format(prediction))

In [17]:
@app.route('/model_health/<model_id>', methods = ['GET'])
def model_health(model_id):
    with open('metrics/model_metrics_version_{0}.json'.format(model_id)) as f:
        model_metrics = json.load(f)    
        return model_metrics